# Connect 4 board class

In [8]:
import numpy as np

'''
Class for the connect 4 game:
Number of rows = 4, Number of columns = 5
The board will be a 2D Numpy array consisting of 0s, 1s, and 2s (where 1 is player 1, 2 is player 2, 0 is an empty slot)
Rewards are as follows: {win: 1, draw: -0.5, lose: -1} (we want to maximize winning)
'''

class C4:
    def __init__(self):
        self.width = 7
        self.height = 6
        self.state = np.zeros([self.height, self.width], dtype=np.uint8)
        self.players = {'P1': 1, 'P2': 2}
        self.rewards = {'Win': 1, 'Draw': -0.5, 'Lose': -1}
        self.Finished = False
        self.actions = [0, 1, 2, 3, 4, 5, 6]
        
    def resetGame(self):
        self.__init__()

    
    '''
    Function for returning the columns which are not full (the topmost slot in the column should be a 0)
    '''

    def free_cols(self):
        return [col for col in range(self.width) if self.state[0, col] == 0]



    '''
    Function for checking winning conditions
    Input will be the player, row & col of move played
    Search for win in the col, row and the two diagonals
    '''
    
    def check_vertical(self, sub_str, col):
        return sub_str in self.state[:, col].astype(str)

    
    def check_horizontal(self, sub_str, row):
        return sub_str in self.state[row, :].astype(str)
    
    def check_diagonal(self, sub_str, row, col):
        left_diagonal = ''

        #first go to the lefmost point in the left diagonal of the row, col
        i = row - min(row, col)
        j = col - min(row, col)
        while i < self.height and j < self.width:
            left_diagonal += f'{self.state[row, col]} '
            i+=1
            j+=1
        
        right_diagonal = ''

        #first go to the rightmost point in the right diagonal of the row, col
        i  = row - min(row, col)
        j = col + min(row, col)
        while i < self.height and j > 0:
            right_diagonal += f'{self.state[row, col]} '
            i+=1
            j-=1

        return sub_str in left_diagonal or sub_str in right_diagonal
    
    #we just need to check if the board is full 
    def is_draw(self):
        for col in range(self.width):
            if self.state[0][col] == 0:
                return False
        return True

    def check_win(self, player, row, col):
        win_substr = ' '.join([self.players[player]] * 4)
        #if either of the conditions passes, the current player has won
        if self.check_vertical(win_substr, col) or self.check_horizontal(win_substr, row) or self.check_diagonal(win_substr, row, col):
            self.Finished = True
        
        if self.Finished:
            return self.rewards['Win']
        elif self.is_draw():
            return self.rewards['Draw']
        else:
            return 0

    '''
    Function for making a move.
    If the move is valid, drop the token at the lowest empty space in the column
    Once the move is made, check winning conditions

    '''

    def move(self, player, col):
        #check if there is free space in the column
        if self.state[0, col] == 0:
            row = np.where(self.state[:, col])[0][-1]
            self.state[row, col] = self.players[player]
            return self.state.copy(), self.check_win(player, row, col)


        else:
            print('Invalid move')
            return self.state.copy(), 0

# Experience Replay Class

In [ ]:
'''
Experience Replay will be used to train.
Any transition that is observed will be stored: (state, action taken, reward received, next state)
We can randomly sample from this list to use for training instead of training on each state-action pair
'''

import random

class Expr_Replay:
    def __init__(self):
        self.store = []

    def sample(self, num):
        return random.sample(self.store, num)
    
    def add(self, transition):
        self.store.append(transition)

    def _len(self):
        return len(self.store)

# Neural Network to approximate the Q table

In [10]:
import torch
from torch import nn
import torch.nn.functional as F

class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()

        #convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(1, 32, kernel_size=5, padding=2)

        #fully connected layers
        self.fc1 = nn.Linear(32 * 6 * 7, 42)
        self.fc2 = nn.Linear(42, 42)
        self.out = nn.Linear(42, 7)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

# Parameters used in training

In [ ]:
import torch.optim as optim
import math

learning_rate = 0.001
batch_size = 200
gamma = 0.9
e_max = 1.0
e_min = 0.01
decay_rate = 0.001
env = C4()
memory = Expr_Replay()
main_net = DQN()
target_net = DQN()
target_net.load_state_dict(main_net.state_dict())
optimizer = optim.Adam(main_net.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()
steps_done = 0
episodes = 25000
target_lag = 15 #update the target net every 15 episodes


#exponential decay used for epsilon
def epsilon_decay(step):
    return e_min + (e_max - e_min) * math.exp(-decay_rate * step)


#add batch size and channel to make it ready for a conv layer
def transform_input(state):
    return torch.tensor(state, dtype=torch.float).view(1, 1, *state.shape)

'''
Function for carrying out epsilon-greedy strategy for the agent
'''
def epsilon_greedy(state, free_actions, step):
    state = transform_input(state)

    threshold = epsilon_decay(step)

    #if less then epsilon, then choose a random available action
    #if greater than epsilon, then use the main NN to exploit
    if random.random() < threshold:
        return random.choice(free_actions)
    else:
        with torch.no_grad():
            actions = main_net(state)[0, :]
            vals = [actions[i] for i in free_actions]
            return free_actions[np.argmax(vals)]

def rand_agent(free_cols):
    return random.choice(free_cols)


def optimize():
    pass

''' 
Function for training our main NN. For each episode, we will randomize who makes the first move so that our agent has experience with both situations.

'''
def train():
    for i in range(episodes):
        env.resetGame()

        #select random player to go first: P1 is always our agent and P2 is always a random agent
        #exposes our agent to episodes where it starts with the second turn
        first = random.choice(env.players)
        if first == 'P2':
            free_actions = env.free_cols()
            a_p2 = rand_agent(free_actions)
            s_p2_curr, reward_p2 = env.move('P2', a_p2)
            s_p1 = s_p2_curr
        else:
            s_p1 = env.state.copy()

        #main loop for each episode
        while True:
            free_actions = env.free_cols()
            a_p1 = epsilon_greedy(s_p1, free_actions, steps_done)
            s_p1_curr, reward_p1 = env.move('P1', a_p1)

            if env.Finished:
                if reward_p1 == 1:
                    memory.add([s_p1, a_p1, 1, None])
                else:
                    memory.add([s_p1, a_p1, -0.5, None])
                break

            free_actions = env.free_cols()
            a_p2 = rand_agent(free_actions)
            s_p2_curr, reward_p2 = env.move("P2", a_p2)

            if env.Finished:
                if reward_p2 == 1:
                    memory.add([s_p1, a_p1, -1, None])
                else:
                    memory.add([s_p1, a_p1, -0.5, None])
                break

            s_p1 = s_p2_curr

            #optimize the main net
            optimize()
        
        if i % target_lag == target_lag - 1:
            target_net.load_state_dict(main_net.state_dict())